In [39]:
from binance.client import Client
import pandas as pd
import numpy as np

In [2]:
api_key = "8AiGAyxlhYQaRpE1s7097hx5sZ12Ogtr8ir9DsyztaD5j24LrI0fEoToDzCI5lle"
api_secret = "VhudTs0HsBVFNdSTghmqfCjUuIXF6rFiXIfROxHIaM71TGgib7NeZ5aOsJUHjI9f"

In [5]:
client = Client(api_key=api_key, api_secret=api_secret)

In [4]:
def get_history(symbol, interval, start, end=None):
    bars = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end, limit=1000)
    
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit="ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "Quote Asset Volume", "Number Of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace=True)

    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors="coerce")
    return df

In [9]:
df = get_history("BTCUSDT", "1h", "1-1-2000")

In [10]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...
2022-11-01 04:00:00,20482.57,20521.87,20466.71,20495.53,8905.815880
2022-11-01 05:00:00,20495.99,20555.21,20491.85,20530.41,8071.529080
2022-11-01 06:00:00,20530.41,20662.80,20525.21,20624.57,14020.497730


In [36]:
class strategy_tester():
    def __init__(self, units=1, tc=0.1):
        self.units = units
        self.tc = tc

    def get_data(self):
        self.data = pd.read_csv(self.filepath, parse_dates=["date"], index_col="date").iloc[::-1]
        
    def get_history(self, symbol, interval, start="1-1-2000", end=None):
        bars = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end, limit=1000)
        
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit="ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "Quote Asset Volume", "Number Of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace=True)

        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors="coerce")
        self.data = df    
    
    def prepare_data(self, sma1, sma2):
        df = self.data[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close.div(df.Close.shift(1)))
        df["sma1"] = df.Close.rolling(sma1).mean()
        df["sma2"] = df.Close.rolling(sma2).mean()
        condition = df.sma1 > df.sma2
        df["position"] = 0
        df.loc[condition, "position"] = 1
    
        self.data = df
    
    def run_backtest(self):
        df = self.data.copy()
        df["strategy"] = df.returns * df.position.shift(1)
        df["trades"] = df.position.diff().fillna(0).abs()  # To calculate fees and shit
        df.strategy = df.strategy + (df.trades * self.tc)  # Calculate fees and shit
        df["creturns"] = df.returns.cumsum().apply(np.exp)
        df["cstrategy"] = df.strategy.cumsum().apply(np.exp)
        self.data = df.copy()
        
    def test_strategy(self, sma1, sma2):
        self.get_history("BTCUSDT", "1h")
        df = self.data
        self.prepare_data(sma1, sma2)
        self.run_backtest()
        
    def optimize_strategy(self, sma1r, sma2r):
        self.datas = []
        performance = []
        combinations = list(product(sma1r, sma2r))
        for comb in combinations:
            self.test_strategy(comb[0], comb[1])
#             self.datas.append(self.data)
            performance.append(self.calc_creturn(self.data))
        self.results_overview =  pd.DataFrame(data=np.array(combinations), columns=["sma1", "sma2"])
        self.results_overview["cstrategy"] = performance
        
    def calc_creturn(self, series):
        return float(series[["strategy"]].sum().apply(np.exp))

In [37]:
trader = strategy_tester()

In [40]:
trader.test_strategy(9, 20)

In [42]:
trader.data

,Close,Volume,returns,sma1,sma2,position,strategy,trades,creturns,cstrategy
Date,,,,,,,,,,
2017-08-17 04:00:00,4308.83,47.181009,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN
2017-08-17 05:00:00,4315.32,23.234916,0.001505,NaN,NaN,0,0.000000,0.0,1.001506,1.000000e+00
2017-08-17 06:00:00,4324.35,7.229691,0.002090,NaN,NaN,0,0.000000,0.0,1.003602,1.000000e+00
2017-08-17 07:00:00,4349.99,4.443249,0.005912,NaN,NaN,0,0.000000,0.0,1.009552,1.000000e+00
2017-08-17 08:00:00,4360.69,0.972807,0.002457,NaN,NaN,0,0.000000,0.0,1.012036,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...
2022-11-01 04:00:00,20495.53,8905.815880,0.000633,20478.513333,20491.2585,0,0.000000,0.0,4.756635,6.304030e+112
2022-11-01 05:00:00,20530.41,8071.529080,0.001700,20492.837778,20482.0785,1,0.100000,1.0,4.764730,6.967031e+112
2022-11-01 06:00:00,20624.57,14020.497730,0.004576,20517.058889,20475.7790,1,0.004576,0.0,4.786582,6.998984e+112


In [46]:
trader.data[["returns", "strategy"]].sum().apply(np.exp)

returns      4.778302e+00
strategy    6.986876e+112
dtype: float64